# Pre generated model: Wurtzite

Here we run "step 4" separately to define a common object for all materials of the same type.

This is useful because each time you run QSYMM to define the kp model, its numerical process may lead Hamiltonian families that are equivalent, but with different numerical prefactors or different order of the $c_n$ coefficients. Therefore, here define the model and store it in a file to be read by the examples that uses this crystal symmetry.

## Three cases

For Wurtzite, the top valence bands transform as the double group irreps G9+G9+G7 of C6v. However, for the conduction band there are two possible cases: G8 or G9.

- For GaN, the first conduction band is G9, and the second is G8.
- For GaP, the first conduction band is G8, and the second is G9.

Therefore, we refer to three models

- H9 (wurtizte9.pickle): the 8x8 model where the conduction band is G9.
- H8 (wurtizte8.pickle): the 8x8 model where the conduction band is G8.
- H10 (wurtizte.pickle): the 10x10 model accounting for two conduction bands G8+G9.

In [1]:
import numpy as np
import pydft2kp as dft2kp
from qsymm.hamiltonian_generator import hamiltonian_from_family
import sympy
import pickle

# Model H10

- Valence = G9+G9+G7
- Conduction = G8+G9
- H is 10x10

In [6]:
# step 4: builds optimal model with qsymm
# s0, sx, sy, sz: Pauli matrices
from pydft2kp.constants import s0, sx, sy, sz
from scipy.linalg import expm, block_diag, inv

# build symmetry operations of C6v⁴ = [C6, Mx]
# consider also T = time-reversal symmetry

#######################################################
# AUXILIARY FUNCTIONS TO BUILD THE SYMMETRY OPERATORS #
#######################################################
# spin rotation matrices, theta = 2pi/n
spinR = lambda n, sj: expm(-1j*(2*np.pi/n/2)*sj)

# ------------------------------------
# Bands: G9c + G8c + G9v + G9v' + G7v'
C6_G9c  = +spinR(6, sz) # S
C6_G8c  = -spinR(6, sz) # B
C6_G9v  = +spinR(6, sz) # Z
C6_G9v1 = +spinR(6, -sz) # (X+iY,up) (X-iY,dw)
C6_G7v1 = np.diag([-1j, 1j]) # (X-iY,up) (X+iY,dw)
U = block_diag(C6_G9c, C6_G8c, C6_G9v, C6_G9v1, C6_G7v1)
C6 = dft2kp.rotation(1/6, [0,0,1], U=U)

Mx_G9c  = 1j*sx
Mx_G8c  = 1j*sx # sign matters? B1 bs B2 from C6v
Mx_G9v  = 1j*sx
Mx_G9v1 = -1j*sx
Mx_G7v1 = -1j*sx
U = block_diag(Mx_G9c, Mx_G8c, Mx_G9v, Mx_G9v1, Mx_G7v1)
Mx = dft2kp.mirror([1,0,0], U=U)

T_G9c  = -1j*sy
T_G8c  = -1j*sy
T_G9v  = -1j*sy
T_G9v1 = -1j*sy
T_G7v1 = -1j*sy
U = block_diag(T_G9c, T_G8c, T_G9v, T_G9v1, T_G7v1)
TRS = dft2kp.time_reversal(3, U=U)
# ------------------------------------

symms = [C6, Mx, TRS]
qs = dft2kp.qsymm(symms, dim=3, total_power=2);

Matrix([
[            c0 + c22*k_x**2 + c22*k_y**2 + c44*k_z**2,                                      I*c9*k_x + c9*k_y,                                                0,      I*c33*k_x**2 - 2*c33*k_x*k_y - I*c33*k_y**2,  c1 + I*c19*k_z + c23*k_x**2 + c23*k_y**2 + c45*k_z**2,     I*c10*k_x + c10*k_y + c37*k_x*k_z - I*c37*k_y*k_z,      I*c11*k_x + c11*k_y + c38*k_x*k_z - I*c38*k_y*k_z, c2 + I*c20*k_z + c24*k_x**2 + c24*k_y**2 + c46*k_z**2,  I*c12*k_x - c12*k_y + c39*k_x*k_z + I*c39*k_y*k_z,         c25*k_x**2 - 2*I*c25*k_x*k_y - c25*k_y**2],
[                                   -I*c9*k_x + c9*k_y,              c0 + c22*k_x**2 + c22*k_y**2 + c44*k_z**2,      I*c33*k_x**2 + 2*c33*k_x*k_y - I*c33*k_y**2,                                                0,     -I*c10*k_x + c10*k_y - c37*k_x*k_z - I*c37*k_y*k_z, c1 + I*c19*k_z + c23*k_x**2 + c23*k_y**2 + c45*k_z**2, -c2 - I*c20*k_z - c24*k_x**2 - c24*k_y**2 - c46*k_z**2,     I*c11*k_x - c11*k_y + c38*k_x*k_z + I*c38*k_y*k_z,         -c25*k_x**2

In [7]:
# save symms and qs object
with open('wurtzite.pickle', 'wb') as fp:
  pickle.dump(symms, fp)
  pickle.dump(qs, fp)

print(len(qs.model))

52


In [8]:
# print model in Latex format
H = hamiltonian_from_family(qs.model, nsimplify=True)

cs = sympy.symbols('c:52')
kx, ky, kz = sympy.symbols('k_x, k_y, k_z')
k2, kp, km = sympy.symbols('k^2, k_+, k_-')
kxy2 = sympy.symbols('k_{\parallel}^2') # = kx**2 + ky**2
Kbig = sympy.symbols('\hat{K}') # = kx**2 - ky**2
# kp = kx+i.ky
# km = kx-i.ky
# kx = (kp+km)/2
# ky = (kp-km)/2i

H = H.applyfunc(lambda ij: sympy.collect(ij, cs))

H = sympy.simplify(H)
H = sympy.simplify(H.subs({kx**2 + ky**2 + kz**2: k2}))
H = sympy.simplify(H.subs({kx**2 + ky**2: kxy2}))
H = sympy.simplify(H.subs({kx+sympy.I*ky: kp, 
                           kx-sympy.I*ky: km,
                           sympy.I*kx-ky: sympy.I*kp, 
                           sympy.I*kx+ky: sympy.I*km,
                           }))

H = sympy.simplify(H.subs({kx**2 - ky**2: Kbig}))

H

Matrix([
[            c0 + c22*k_{\parallel}^2 + c44*k_z**2,                                           I*c9*k_-,                                      0,  c33*(I*k_x**2 - 2*k_x*k_y - I*k_y**2),  c1 + I*c19*k_z + c23*k_{\parallel}^2 + c45*k_z**2,                             k_-*(I*c10 + c37*k_z),                              k_-*(I*c11 + c38*k_z), c2 + I*c20*k_z + c24*k_{\parallel}^2 + c46*k_z**2,                 k_+*(I*c12 + c39*k_z),            c25*(\hat{K} - 2*I*k_x*k_y)],
[                                        -I*c9*k_+,              c0 + c22*k_{\parallel}^2 + c44*k_z**2,  c33*(I*k_x**2 + 2*k_x*k_y - I*k_y**2),                                      0,                             -k_+*(I*c10 + c37*k_z), c1 + I*c19*k_z + c23*k_{\parallel}^2 + c45*k_z**2, -c2 - I*c20*k_z - c24*k_{\parallel}^2 - c46*k_z**2,                             k_+*(I*c11 + c38*k_z),          -c25*(\hat{K} + 2*I*k_x*k_y),                  k_-*(I*c12 + c39*k_z)],
[                                                0,

In [9]:
print(sympy.latex(H))

\left[\begin{matrix}c_{0} + c_{22} k_{\parallel}^2 + c_{44} k_{z}^{2} & i c_{9} k_{-} & 0 & c_{33} \left(i k_{x}^{2} - 2 k_{x} k_{y} - i k_{y}^{2}\right) & c_{1} + i c_{19} k_{z} + c_{23} k_{\parallel}^2 + c_{45} k_{z}^{2} & k_{-} \left(i c_{10} + c_{37} k_{z}\right) & k_{-} \left(i c_{11} + c_{38} k_{z}\right) & c_{2} + i c_{20} k_{z} + c_{24} k_{\parallel}^2 + c_{46} k_{z}^{2} & k_{+} \left(i c_{12} + c_{39} k_{z}\right) & c_{25} \left(\hat{K} - 2 i k_{x} k_{y}\right)\\- i c_{9} k_{+} & c_{0} + c_{22} k_{\parallel}^2 + c_{44} k_{z}^{2} & c_{33} \left(i k_{x}^{2} + 2 k_{x} k_{y} - i k_{y}^{2}\right) & 0 & - k_{+} \left(i c_{10} + c_{37} k_{z}\right) & c_{1} + i c_{19} k_{z} + c_{23} k_{\parallel}^2 + c_{45} k_{z}^{2} & - c_{2} - i c_{20} k_{z} - c_{24} k_{\parallel}^2 - c_{46} k_{z}^{2} & k_{+} \left(i c_{11} + c_{38} k_{z}\right) & - c_{25} \left(\hat{K} + 2 i k_{x} k_{y}\right) & k_{-} \left(i c_{12} + c_{39} k_{z}\right)\\0 & c_{33} \left(- i k_{x}^{2} + 2 k_{x} k_{y} + i k_{y}^{2}

# Model H8

- Valence = G9+G9+G7
- Conduction = G8
- H is 8x8

In [10]:
U = block_diag(C6_G8c, C6_G9v, C6_G9v1, C6_G7v1)
C6 = dft2kp.rotation(1/6, [0,0,1], U=U)

U = block_diag(Mx_G8c, Mx_G9v, Mx_G9v1, Mx_G7v1)
Mx = dft2kp.mirror([1,0,0], U=U)

U = block_diag(T_G8c, T_G9v, T_G9v1, T_G7v1)
TRS = dft2kp.time_reversal(3, U=U)
# ------------------------------------

symms = [C6, Mx, TRS]
qs = dft2kp.qsymm(symms, dim=3, total_power=2);

Matrix([
[       c0 + c13*k_x**2 + c13*k_y**2 + c27*k_z**2,                                I*c6*k_x + c6*k_y,                                                      0,           I*c20*k_x**2 - 2*c20*k_x*k_y - I*c20*k_y**2,            I*c21*k_x**2 - 2*c21*k_x*k_y - I*c21*k_y**2,                                                     0,        I*c22*k_x**2 + 2*c22*k_x*k_y - I*c22*k_y**2,   I*c26*k_x*k_z - c26*k_y*k_z + c5*k_x + I*c5*k_y],
[                              -I*c6*k_x + c6*k_y,        c0 + c13*k_x**2 + c13*k_y**2 + c27*k_z**2,            I*c20*k_x**2 + 2*c20*k_x*k_y - I*c20*k_y**2,                                                     0,                                                      0,          -I*c21*k_x**2 - 2*c21*k_x*k_y + I*c21*k_y**2,    I*c26*k_x*k_z + c26*k_y*k_z + c5*k_x - I*c5*k_y,      -I*c22*k_x**2 + 2*c22*k_x*k_y + I*c22*k_y**2],
[                                               0,     -I*c20*k_x**2 + 2*c20*k_x*k_y + I*c20*k_y**2,              c1 + c14*k_x**2 + c14*k

In [11]:
# save symms and qs object
with open('wurtzite8.pickle', 'wb') as fp:
  pickle.dump(symms, fp)
  pickle.dump(qs, fp)
print(len(qs.model))

32


In [12]:
# print model in Latex format
H = hamiltonian_from_family(qs.model, nsimplify=True)

cs = sympy.symbols('c:32')
kx, ky, kz = sympy.symbols('k_x, k_y, k_z')
k2, kp, km = sympy.symbols('k^2, k_+, k_-')
kxy2 = sympy.symbols('k_{\parallel}^2') # = kx**2 + ky**2
Kbig = sympy.symbols('\hat{K}') # = kx**2 - ky**2
# kp = kx+i.ky
# km = kx-i.ky
# kx = (kp+km)/2
# ky = (kp-km)/2i

H = H.applyfunc(lambda ij: sympy.collect(ij, cs))

H = sympy.simplify(H)
H = sympy.simplify(H.subs({kx**2 + ky**2 + kz**2: k2}))
H = sympy.simplify(H.subs({kx**2 + ky**2: kxy2}))
H = sympy.simplify(H.subs({kx+sympy.I*ky: kp, 
                           kx-sympy.I*ky: km,
                           sympy.I*kx-ky: sympy.I*kp, 
                           sympy.I*kx+ky: sympy.I*km,
                           }))

H = sympy.simplify(H.subs({kx**2 - ky**2: Kbig}))

H

Matrix([
[ c0 + c13*k_{\parallel}^2 + c27*k_z**2,                               I*c6*k_-,                                                  0,             c20*(I*k_x**2 - 2*k_x*k_y - I*k_y**2),              c21*(I*k_x**2 - 2*k_x*k_y - I*k_y**2),                                                 0, c22*(I*k_x**2 + 2*k_x*k_y - I*k_y**2),                   k_+*(I*c26*k_z + c5)],
[                             -I*c6*k_+,  c0 + c13*k_{\parallel}^2 + c27*k_z**2,              c20*(I*k_x**2 + 2*k_x*k_y - I*k_y**2),                                                 0,                                                  0,            c21*(-I*k_x**2 - 2*k_x*k_y + I*k_y**2),                  k_-*(I*c26*k_z + c5), c22*(-I*k_x**2 + 2*k_x*k_y + I*k_y**2)],
[                                     0, c20*(-I*k_x**2 + 2*k_x*k_y + I*k_y**2),              c1 + c14*k_{\parallel}^2 + c28*k_z**2,                                          I*c7*k_-,                               k_-*(c23*k_z + I*c8), I*c12*k_z + c15*k_{\p

In [13]:
print(sympy.latex(H))

\left[\begin{matrix}c_{0} + c_{13} k_{\parallel}^2 + c_{27} k_{z}^{2} & i c_{6} k_{-} & 0 & c_{20} \left(i k_{x}^{2} - 2 k_{x} k_{y} - i k_{y}^{2}\right) & c_{21} \left(i k_{x}^{2} - 2 k_{x} k_{y} - i k_{y}^{2}\right) & 0 & c_{22} \left(i k_{x}^{2} + 2 k_{x} k_{y} - i k_{y}^{2}\right) & k_{+} \left(i c_{26} k_{z} + c_{5}\right)\\- i c_{6} k_{+} & c_{0} + c_{13} k_{\parallel}^2 + c_{27} k_{z}^{2} & c_{20} \left(i k_{x}^{2} + 2 k_{x} k_{y} - i k_{y}^{2}\right) & 0 & 0 & c_{21} \left(- i k_{x}^{2} - 2 k_{x} k_{y} + i k_{y}^{2}\right) & k_{-} \left(i c_{26} k_{z} + c_{5}\right) & c_{22} \left(- i k_{x}^{2} + 2 k_{x} k_{y} + i k_{y}^{2}\right)\\0 & c_{20} \left(- i k_{x}^{2} + 2 k_{x} k_{y} + i k_{y}^{2}\right) & c_{1} + c_{14} k_{\parallel}^2 + c_{28} k_{z}^{2} & i c_{7} k_{-} & k_{-} \left(c_{23} k_{z} + i c_{8}\right) & i c_{12} k_{z} + c_{15} k_{\parallel}^2 + c_{2} + c_{29} k_{z}^{2} & k_{+} \left(c_{24} k_{z} + i c_{9}\right) & c_{16} \left(\hat{K} - 2 i k_{x} k_{y}\right)\\c_{20} \le

# Model H9

- Valence = G9+G9+G7
- Conduction = G9
- H is 8x8

In [14]:
U = block_diag(C6_G9c, C6_G9v, C6_G9v1, C6_G7v1)
C6 = dft2kp.rotation(1/6, [0,0,1], U=U)

U = block_diag(Mx_G9c, Mx_G9v, Mx_G9v1, Mx_G7v1)
Mx = dft2kp.mirror([1,0,0], U=U)

U = block_diag(T_G9c, T_G9v, T_G9v1, T_G7v1)
TRS = dft2kp.time_reversal(3, U=U)
# ------------------------------------

symms = [C6, Mx, TRS]
qs = dft2kp.qsymm(symms, dim=3, total_power=2);

Matrix([
[             c0 + c19*k_x**2 + c19*k_y**2 + c35*k_z**2,                                     I*c7*k_x + c7*k_y,  c1 + I*c16*k_z + c20*k_x**2 + c20*k_y**2 + c36*k_z**2,       c29*k_x*k_z - I*c29*k_y*k_z + I*c8*k_x + c8*k_y,        c30*k_x*k_z - I*c30*k_y*k_z + I*c9*k_x + c9*k_y, I*c17*k_z + c2 + c21*k_x**2 + c21*k_y**2 + c37*k_z**2,  I*c10*k_x - c10*k_y + c31*k_x*k_z + I*c31*k_y*k_z,         c22*k_x**2 - 2*I*c22*k_x*k_y - c22*k_y**2],
[                                    -I*c7*k_x + c7*k_y,             c0 + c19*k_x**2 + c19*k_y**2 + c35*k_z**2,       -c29*k_x*k_z - I*c29*k_y*k_z - I*c8*k_x + c8*k_y, c1 + I*c16*k_z + c20*k_x**2 + c20*k_y**2 + c36*k_z**2, -I*c17*k_z - c2 - c21*k_x**2 - c21*k_y**2 - c37*k_z**2,       c30*k_x*k_z + I*c30*k_y*k_z + I*c9*k_x - c9*k_y,         -c22*k_x**2 - 2*I*c22*k_x*k_y + c22*k_y**2, I*c10*k_x + c10*k_y + c31*k_x*k_z - I*c31*k_y*k_z],
[ c1 - I*c16*k_z + c20*k_x**2 + c20*k_y**2 + c36*k_z**2,      -c29*k_x*k_z + I*c29*k_y*k_z + I*c8*k_x + c8*k_y,    

In [15]:
# save symms and qs object
with open('wurtzite9.pickle', 'wb') as fp:
  pickle.dump(symms, fp)
  pickle.dump(qs, fp)
print(len(qs.model))

42


In [16]:
# print model in Latex format
H = hamiltonian_from_family(qs.model, nsimplify=True)

cs = sympy.symbols('c:42')
kx, ky, kz = sympy.symbols('k_x, k_y, k_z')
k2, kp, km = sympy.symbols('k^2, k_+, k_-')
kxy2 = sympy.symbols('k_{\parallel}^2') # = kx**2 + ky**2
Kbig = sympy.symbols('\hat{K}') # = kx**2 - ky**2
# kp = kx+i.ky
# km = kx-i.ky
# kx = (kp+km)/2
# ky = (kp-km)/2i

H = H.applyfunc(lambda ij: sympy.collect(ij, cs))

H = sympy.simplify(H)
H = sympy.simplify(H.subs({kx**2 + ky**2 + kz**2: k2}))
H = sympy.simplify(H.subs({kx**2 + ky**2: kxy2}))
H = sympy.simplify(H.subs({kx+sympy.I*ky: kp, 
                           kx-sympy.I*ky: km,
                           sympy.I*kx-ky: sympy.I*kp, 
                           sympy.I*kx+ky: sympy.I*km,
                           }))

H = sympy.simplify(H.subs({kx**2 - ky**2: Kbig}))

H

Matrix([
[             c0 + c19*k_{\parallel}^2 + c35*k_z**2,                                          I*c7*k_-,  c1 + I*c16*k_z + c20*k_{\parallel}^2 + c36*k_z**2,                              k_-*(c29*k_z + I*c8),                               k_-*(c30*k_z + I*c9), I*c17*k_z + c2 + c21*k_{\parallel}^2 + c37*k_z**2,                 k_+*(I*c10 + c31*k_z),           c22*(\hat{K} - 2*I*k_x*k_y)],
[                                         -I*c7*k_+,             c0 + c19*k_{\parallel}^2 + c35*k_z**2,                              -k_+*(c29*k_z + I*c8), c1 + I*c16*k_z + c20*k_{\parallel}^2 + c36*k_z**2, -I*c17*k_z - c2 - c21*k_{\parallel}^2 - c37*k_z**2,                              k_+*(c30*k_z + I*c9),          -c22*(\hat{K} + 2*I*k_x*k_y),                 k_-*(I*c10 + c31*k_z)],
[ c1 - I*c16*k_z + c20*k_{\parallel}^2 + c36*k_z**2,                             k_-*(-c29*k_z + I*c8),              c23*k_{\parallel}^2 + c3 + c38*k_z**2,                                         I*c11*k_-,       

In [18]:
print(sympy.latex(H))

\left[\begin{matrix}c_{0} + c_{19} k_{\parallel}^2 + c_{35} k_{z}^{2} & i c_{7} k_{-} & c_{1} + i c_{16} k_{z} + c_{20} k_{\parallel}^2 + c_{36} k_{z}^{2} & k_{-} \left(c_{29} k_{z} + i c_{8}\right) & k_{-} \left(c_{30} k_{z} + i c_{9}\right) & i c_{17} k_{z} + c_{2} + c_{21} k_{\parallel}^2 + c_{37} k_{z}^{2} & k_{+} \left(i c_{10} + c_{31} k_{z}\right) & c_{22} \left(\hat{K} - 2 i k_{x} k_{y}\right)\\- i c_{7} k_{+} & c_{0} + c_{19} k_{\parallel}^2 + c_{35} k_{z}^{2} & - k_{+} \left(c_{29} k_{z} + i c_{8}\right) & c_{1} + i c_{16} k_{z} + c_{20} k_{\parallel}^2 + c_{36} k_{z}^{2} & - i c_{17} k_{z} - c_{2} - c_{21} k_{\parallel}^2 - c_{37} k_{z}^{2} & k_{+} \left(c_{30} k_{z} + i c_{9}\right) & - c_{22} \left(\hat{K} + 2 i k_{x} k_{y}\right) & k_{-} \left(i c_{10} + c_{31} k_{z}\right)\\c_{1} - i c_{16} k_{z} + c_{20} k_{\parallel}^2 + c_{36} k_{z}^{2} & k_{-} \left(- c_{29} k_{z} + i c_{8}\right) & c_{23} k_{\parallel}^2 + c_{3} + c_{38} k_{z}^{2} & i c_{11} k_{-} & k_{-} \left(i c_